# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


### Spark SQL

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 918.3 kB/s eta 0:00:0000:0100:06
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 824.4 kB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425347 sha256=38778041d6a36e8d307887e87a295611087dd0924c4272d0b76aafaae3fd63f2
  Stored in directory: /home/jovyan/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-21 11:14:20,166 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [3]:
data = se.read.csv('hdfs:/data/clickstream.csv', header=True, sep='\t')

In [4]:
data.show(5)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows



In [5]:
data.createOrReplaceTempView('clickstream')

In [33]:
se.sql("""
with error as -- find lines with errors
(
select  user_id, session_id, event_page, timestamp, 
sum(case when event_type like '%error%' then 1 else 0 end) over (partition by user_id, session_id order by timestamp) flag_sum
from clickstream
),

eq_next_row as -- find rows with duplicates
(
select
*, 
case 
    when lag(event_page) over (partition by user_id, session_id order by timestamp) = event_page then 1 
    else 0 
end as duplicate_row
from error
where flag_sum = 0
),

path as --obtaining paths
( 
select *, concat_ws('-', list_row) path
from 
(
select user_id, session_id, event_page, timestamp, collect_list(event_page) over (partition by user_id, session_id order by timestamp) list_row
from eq_next_row
where duplicate_row = 0
group by user_id, session_id, event_page, timestamp) t
)
select  full_path route, count(*) count -- 
from 
(
select user_id, session_id, max(path) as full_path
from path
group by user_id, session_id
) t
group by
full_path
order by count desc
limit 30
""").show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



### Spark DF

In [62]:
# removing rows with errors
error = data.withColumn('flag', F.sum(F.when(F.col('event_type').like('%error%'), 1).otherwise(0)).over(Window.partitionBy('user_id', 'session_id').orderBy('timestamp')))
filtered_data_error = error.filter(F.col('flag') == 0)
filtered_data_error.show(10)

+-------+----------+----------+----------+----------+----+
|user_id|session_id|event_type|event_page| timestamp|flag|
+-------+----------+----------+----------+----------+----+
|      1|      1026|      page|      main|1700148765|   0|
|     10|       762|      page|      main|1699442253|   0|
|     10|       762|      page|  internet|1699442893|   0|
|     10|       762|     event|  internet|1699443462|   0|
|     10|       762|     event|  internet|1699444244|   0|
|     10|       762|     event|  internet|1699445640|   0|
|     10|       762|     event|  internet|1699449575|   0|
|     10|       762|     event|  internet|1699450822|   0|
|     10|       762|      page|   archive|1699450919|   0|
|     10|       762|      page|     bonus|1699451247|   0|
+-------+----------+----------+----------+----------+----+
only showing top 10 rows



In [63]:
# Remove paths with duplicates, for example, bonus-bonus
filtered_data_dupplicate = filtered_data_error.withColumn('dupl', F.when(F.lag('event_page').over(Window.partitionBy('user_id', 'session_id').orderBy('timestamp')) == F.col('event_page'), 1).otherwise(0)).filter(F.col('dupl') == 0)
filtered_data_dupplicate.show(10)

+-------+----------+----------+----------+----------+----+----+
|user_id|session_id|event_type|event_page| timestamp|flag|dupl|
+-------+----------+----------+----------+----------+----+----+
|      1|      1026|      page|      main|1700148765|   0|   0|
|     10|       762|      page|      main|1699442253|   0|   0|
|     10|       762|      page|  internet|1699442893|   0|   0|
|     10|       762|      page|   archive|1699450919|   0|   0|
|     10|       762|      page|     bonus|1699451247|   0|   0|
|     10|       762|      page|  internet|1699451842|   0|   0|
|     10|       762|      page|      main|1699453927|   0|   0|
|     10|       762|      page|    rabota|1699454103|   0|   0|
|     10|       762|      page|      main|1699457851|   0|   0|
|     10|       762|      page|   archive|1699459270|   0|   0|
+-------+----------+----------+----------+----------+----+----+
only showing top 10 rows



In [64]:
# obtaining list of paths
list_rows = filtered_data_dupplicate.withColumn('list', F.collect_list('event_page').over(Window.partitionBy('user_id', 'session_id').orderBy('timestamp')))
list_rows.show(10)                                                    

+-------+----------+----------+----------+----------+----+----+--------------------+
|user_id|session_id|event_type|event_page| timestamp|flag|dupl|                list|
+-------+----------+----------+----------+----------+----+----+--------------------+
|      1|      1026|      page|      main|1700148765|   0|   0|              [main]|
|     10|       762|      page|      main|1699442253|   0|   0|              [main]|
|     10|       762|      page|  internet|1699442893|   0|   0|    [main, internet]|
|     10|       762|      page|   archive|1699450919|   0|   0|[main, internet, ...|
|     10|       762|      page|     bonus|1699451247|   0|   0|[main, internet, ...|
|     10|       762|      page|  internet|1699451842|   0|   0|[main, internet, ...|
|     10|       762|      page|      main|1699453927|   0|   0|[main, internet, ...|
|     10|       762|      page|    rabota|1699454103|   0|   0|[main, internet, ...|
|     10|       762|      page|      main|1699457851|   0|   0|[m

In [65]:
# obtaining the desired path format
string_paths = list_rows.withColumn('path', F.concat_ws('-', F.col('list')))

In [67]:
# leaving only the final paths
correct_paths = string_paths.groupby('user_id', 'session_id').agg(F.max('path').alias('route'))
correct_paths.show(10)

+-------+----------+--------------------+
|user_id|session_id|               route|
+-------+----------+--------------------+
|      1|      1026|                main|
|     10|       762|main-internet-arc...|
|   1001|        11|main-bonus-intern...|
|   1002|       285|main-news-interne...|
|   1002|       291|        main-archive|
|   1003|       563|main-internet-mai...|
|   1003|       593|main-internet-onl...|
|   1004|       871|main-archive-vkla...|
|   1006|       575|main-bonus-archiv...|
|   1007|        82|main-archive-digital|
+-------+----------+--------------------+
only showing top 10 rows



In [71]:
result = correct_paths.groupby('route').agg(F.count('*').alias('count')).orderBy(F.desc('count')).limit(30)
result.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



2024-01-21 18:12:49,953 WARN spark.HeartbeatReceiver: Removing executor 2 with no recent heartbeats: 10898145 ms exceeds timeout 120000 ms
2024-01-21 18:12:50,247 WARN spark.HeartbeatReceiver: Removing executor 1 with no recent heartbeats: 10897272 ms exceeds timeout 120000 ms
2024-01-21 18:12:54,002 ERROR netty.Inbox: Ignoring error
java.lang.AssertionError: assertion failed: BlockManager re-registration shouldn't succeed when the executor is lost
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$register(BlockManagerMasterEndpoint.scala:587)
	at org.apache.spark.storage.BlockManagerMasterEndpoint$$anonfun$receiveAndReply$1.applyOrElse(BlockManagerMasterEndpoint.scala:119)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:103)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.